To evaluate the effectiveness of the RAG model, we need a dataset and an evaluation method. Luckily, DeepEval provides a way to automate creating the dataset. Unfortunately, it is designed for use with text. Therefore, we will have to do something else to evaluate the images. 

In [9]:
import pandas as pd

from deepeval.synthesizer import Synthesizer
from deepeval.synthesizer.config import StylingConfig
from deepeval.models import DeepEvalBaseLLM

import transformers
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

We load the metadata sample that we used for the language detection test in language_detection.ipynb and for the product type check with Llama-vision in Huggingface_Llama_Vision.ipynb.

In [10]:
pdf_sample = pd.read_pickle('/mnt/d/abo-dataset/abo-listings-sample.pkl')
pdf_sample = pdf_sample[['item_name', 'brand', 'model_name', 'model_year', 
                     'product_description', 'product_type', 'color',
                     'fabric_type', 'style', 'material', 'item_keywords',
                     'pattern', 'finish_type', 'bullet_point']]

pdf_sample['product_type'] = pdf_sample['product_type'].str.replace('_', ' ')
pdf_sample.loc[pdf_sample['product_type'] == 'FINERING', 'product_type'] = 'FINE RING'
pdf_sample.loc[pdf_sample['product_type'] == 'FINENECKLACEBRACELETANKLET', 'product_type'] = 'FINE NECKLACE BRACELET ANKLET'
pdf_sample.loc[pdf_sample['product_type'] == 'FINEEARRING', 'product_type'] = 'FINE EARRING'
pdf_sample.loc[pdf_sample['product_type'] == 'FASHIONNECKLACEBRACELETANKLET', 'product_type'] = 'FASHION NECKLACE BRACELET ANKLET'
pdf_sample.loc[pdf_sample['product_type'] == 'FINEOTHER', 'product_type'] = 'FINE OTHER'
pdf_sample.loc[pdf_sample['product_type'] == 'FASHIONEARRING', 'product_type'] = 'FASHION EARRING'
pdf_sample.loc[pdf_sample['product_type'] == 'SHOWERHEAD', 'product_type'] = 'SHOWER HEAD'
pdf_sample.loc[pdf_sample['product_type'] == 'FASHIONOTHER', 'product_type'] = 'FASHION OTHER'
pdf_sample['product_type'] = pdf_sample['product_type'].str.replace('ABIS ', '')

The LLMs take strings, so convert each row to one.

In [11]:
def row_to_str(row):
    row_filtered = row.dropna()
    text = []
    for row_item in row_filtered:
        if isinstance(row_item, list):
            for list_item in row_item:
                text.append(str(list_item) + ';')
        else:
            text.append(str(row_item) + ';')
            
    final_string = ' '.join(text).replace('\n', ' ').replace('^', ' ').replace(',', ', ')
    return final_string[:-1]

In [12]:
row_strings = []
for item_id in pdf_sample.index:
    row = pdf_sample.loc[item_id]
    row_strings.append(row_to_str(row))

In [13]:
row_strings

['Amazon Brand:Belei Moisturizing Lifting Eye Mask with Diamond Powder Pack of 5; BEAUTY; With this firming eye mask,  your skin will look youthful again. The active ingredients help to protect and moisturise the skin.; Contains diamond powder - an active ingredient with lifting effect and a firming and brightening effect for more radiance; This mineral for external use makes the skin shine and youthful; Contains hyaluronic acid - a moisturising film that provides moisture and elasticity; The skin is refreshed and wrinkles are reduced.; Contains Coenzyme Q10 - with strong antioxidant properties that help protect the skin from the visible signs of ageing.; The skin is gently nourished and has a fresh,  healthy and radiant.',
 'Amazon Brand - Belei - Lifting Sheet Mask with Hyaluronic Acid,  Pack of 10; BELEI; BEAUTY; Lifting; peel off face masks; serum; other; multimask; moisturiser face women; face mask; aloe vera; face cleanser; face masks beauty; face masks; face masks skincare; shee

In [14]:
class CustomLlama31_8B(DeepEvalBaseLLM):
    def __init__(self):
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True
        )

        model_8bit = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct",
            device_map="cuda",
            quantization_config=quantization_config,
        )
        tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct"
        )

        self.model = model_8bit
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        pipeline = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            use_cache=True,
            device_map="auto",
            max_length=2500,
            do_sample=True,
            top_k=5,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )

        return pipeline(prompt)

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Llama-3.1 8B"

We need to tell the synthesizer how our RAG system is to be used.

In [15]:
styling_config = StylingConfig(
    input_format="An English-language request for a product the user desires.",
    expected_output_format="A product name with a brief description.",
    task="A useful shopping assistant that helps users find products that meet their specifications.",
    scenario="Non-technical users looking to do some shopping without the usual hassle of looking through lists of products."
    )

Run the synthesizer.

In [16]:
synthesizer = Synthesizer(model=CustomLlama31_8B(), styling_config=styling_config)
goldens = synthesizer.generate_goldens_from_contexts(contexts=[[row_string] for row_string in row_strings])

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-675badf1-5817a32441dd1dec0acc34b4;b33e4ae5-055c-4f27-9d52-0c05001a145d)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.